# AI absolute basics using Python

based on material from Joel Grus, adapted by Heiko Rölke

Now that we have learned ourselves some Python and libraries, we will look at some very basic (read: boring) AI stuff. Later, in hindsight, it will be surprising how simple the basic building blocks of Deep Learning are.

In [ ]:
# some simple loss function

def sum_of_squares(vect):
    return sum([x**2 for x in vect])

We are looking for the input vector "vect" that minimizes the return result.

I know, this is dead easy to come up with, but let's assume we don't know the correct answer.
What can we do?

We choose a random start point (vector) and look for the *gradient*, the vector of partial derivatives. The gradient tells us in which direction to go to come closer to a maximum - and therefor also to a minimum. We choose a step width, apply this step to the input vector and start anew. Until the return value does not change (much) anymore - then we have reached a minimum. Possibly a local one... 

Let's further assume that we also do not know the derivative function of our loss function. To come up with an approximation, we use the asymptotic difference quotient - you might recall this from math in school or university(?)

In [ ]:
def difference_quotient(func, x, h):
    return (func(x+h) - func(x)) / h

We can easily test this for a simple example, to see that we can come reasonably close to the actual values:

In [ ]:
def square(x):
    return x*x

# you might recall that the derivative of x^2 is 2x
def derivative(x):
    return 2*x

In [ ]:
xs = range(-10,11)
actuals = [derivative(x) for x in xs]
estimates = [difference_quotient(square, x, h=0.001) for x in xs]

from matplotlib import pyplot as plt
plt.title("Actual Derivatives vs. Estimates")
plt.plot(xs, actuals, "rx", label="Actual")
plt.plot(xs, estimates, "b+", label="Estimate")
plt.legend(loc=9)
plt.show()

Fits reasonably well...
No Proof! But some reassurance.

In [ ]:
import numpy as np

# i-th partial "derivative" of func over input vect with distance h 
def partial_difference_quotient(func, vect, i, h):
    w = [v_j + (h if j==i else 0) for j, v_j in enumerate(vect)]
    return (func(w) - func(vect)) / h


# estimate gradient using partial difference quotient
def estimate_gradient(func, vect, h=0.0001):
    return np.array([partial_difference_quotient(func, vect, i, h) for i in range(len(vect))])

We now want to put this into practice. We use a three-dimensional vector for the sum-of-squares function above and want to find the minimum - all zeros.

For that, we choose a random starting vector, calculate the gradient and make a small step in the opposite direction. This is repeated for a fixed amount of steps (or until the gradient becomes very small).

In [ ]:
def gradient_step(vect, grad, step_size):
    step = step_size * grad
    return vect + step


# choose random starting point
vect = np.random.uniform(-10,10,3)

for epoch in range(1000):
    grad = estimate_gradient(sum_of_squares, vect) 
    vect = gradient_step(vect, grad, -0.01)
    print(epoch, vect)

# Some remarks on efficiency

The method we learned here works for nearly every case (except some really weird loss functions). 

But: Those of you concerned about code efficiency (I guess all of you) might have noticed that we introduced a lot of calculations. For each gradient step and a vector of length n we have to calculate the loss function 2n times. Thats a lot of work considering modern LLMs do have billions of parameters (= loss function calculations per step) and are trained "countless" times (think number of inputs (billions to trillions of tokens) times training steps (easily hundreds to thousands of so-called "epochs").

There are some simple and many not so simple optimisations. Unfortunately, we cannot introduce them here. But maybe you can come up with some suggestions?  

## From here on: (mostly) differentiable functions

In ML, especially DL, we prefer loss functions that are differentiable, so that we do not have to use the difference quotient each time. But be assured this would be possible, as you can easily try out for yourself. 

Sometimes we will use functions that are not differentiable, or at least not everywhere. This will be clearly stated.

# An application example: fitting functions on data

In practice, we of have data and want to find a function fitting this data. 

A simple example: We have some data points and suspect that a linear function could fit the data. We want to find the parameters of that function (slope, intercept) so that the error - the distance of data points to the function - is minimized.

For the error function, we use the mean quadratic distance of each data point to the function approximation. 

In [ ]:
# generate data for the linear function f(x) = 20*x + 5
inputs = np.array([(x, 20*x+5) for x in range(-50,50)])

In [ ]:
# gradient method for linear function approximation
def linear_gradient(x, y, theta):  # theta is the vector of the (current estimates of the) function coefficients
    slope, intercept = theta
    predicted = slope * x + intercept
    error = predicted - y
    grad_slope = 2 * error * x  # "cheated", as explained above - this is the partial derivation of the error function for the slope
    grad_intercept = 2 * error   # and for the intercept (no x)
    return np.array([grad_slope, grad_intercept])

In [ ]:
theta = np.random.uniform(-1,1,2)  # random values as a starter

learning_rate = 0.001  # a new name for the value h, the step size

for epoch in range(5000):
    grad = np.mean(np.array([linear_gradient(x, y, theta) for x, y in inputs]), axis=0)
    theta = gradient_step(theta, grad, -learning_rate)
    print(epoch, theta)

### Batch or Minibatch?

What we did so far was to use all data in all steps. Feasible for our toy example, but impractical for larger problems.

One solution is to split the complete data batch to so-called mini-batches and train on those:

In [ ]:
import random

def minibatches(dataset, batch_size, shuffle = True):
    # Start index 0, batch_size, 2 * batch_size, ...
    batch_starts = [start for start in range(0, len(dataset), batch_size)]
    if shuffle: random.shuffle(batch_starts) # shuffle batches
    for start in batch_starts:
        end = start + batch_size
        yield dataset[start:end]

In [ ]:
# Gradient Descent using Minibatches
theta = np.random.uniform(-1,1,2)  # random values as a starter

for epoch in range(1000):
    for batch in minibatches(inputs, batch_size=20):
        grad = np.mean(np.array([linear_gradient(x, y, theta) for x, y in batch]), axis=0)
        theta = gradient_step(theta, grad, -learning_rate)
        print(epoch, theta)

### Minibatches to the extremes: Stochastic Gradient Descent (SGD)

Why not using smaller minibatches? Batch size 1 might be sufficient...

In [ ]:
# Stochastic Gradient Descent
theta = np.random.uniform(-1,1,2)  # random values as a starter

for epoch in range(100):
    for x, y in inputs:
        grad = linear_gradient(x, y, theta)
        theta = gradient_step(theta, grad, -learning_rate)
        print(epoch, theta)

For this example, the SGD is faster than the other methods, as calculating the mean error is bad for fast approximation. For other, less perfect circumstances this might be the other way round. 

# Exercises

Before we go to the next section, we want to play around with the concepts above.

1. For the code up to the cell with the "gradient_step" function. Make sure the gradient descent code is working for you. Play around with:
    * step sizes
    * functions
    * number of epochs
    * ...
2. Visualize the learning process
    * for the original code
    * for your experiments
3. Have a look at the function fitting example and make sure it works for you as well.
    * Change the function parameter, generate new data and find a fitting function for that.
    * *Advanced*: Change the example to a non-linear function, e.g. a quadratic (or higher) function.
    * Visualize your data and (some of the) approximated functions